In [ ]:
import pandas as pd

df_test = pd.read_csv("vuln_data_test.csv")
df_test = df_test.groupby("vuln_code").agg({
    "vuln_title": list,  # Collect all vuln_titles in a list
    "vuln_explanation": list  # Optional: collect explanations too
}).reset_index()

def make_explanation(row):
    all_explanation = []
    for item in row['vuln_explanation']:
        all_explanation.append(item)
    ctx = "\n\n".join([x for x in all_explanation])
    return ctx

df_test['full_context'] = df_test.apply(make_explanation, axis=1)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "codellama/CodeLlama-34b-Instruct-hf",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


INFO 05-13 04:38:15 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-13 04:38:15 [__init__.py:239] Automatically detected platform cuda.


2025-05-13 04:38:17,099	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.5.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.749 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 7/7 [03:32<00:00, 30.36s/it]


codellama/CodeLlama-34b-Instruct-hf does not have a padding token! Will use pad_token = <unk>.


In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train.csv',
    'val': 'vuln_data_val.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

import random

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world.""",
]

PROMPTS = [
    """The given Solidity function is vulnerable. Review the code and analyze its security flaws. Just give short explanation why these function is vulnerable.
    
    This is the function we need to audit:
    ```solidity
    {code}
    ```
    
Vulnerability : """,
]


def make_conversational(examples):
    code = examples['vuln_code']
    desc = examples['vuln_explanation']
    assistant_prompt = f"""{desc}"""
    selected_system_prompt = random.choice(SYSTEM_PROMPT)
    selected_user_prompt = random.choice(PROMPTS) 
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code)},
                    {"role": "assistant", "content": assistant_prompt}]
    return { "conversations" : conversation, }

dataset = dataset.map(make_conversational)
dataset = dataset.remove_columns(["vuln_title", "vuln_explanation", "severity", "vuln_recommendation"])

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

test_dataset = dataset['test']
test_dataset = test_dataset.map(lambda row: {'answer':row['conversations'][-1]})
test_dataset = test_dataset.map(lambda row: {'conversations':row['conversations'][:-1]})

In [ ]:
import re
from unsloth.chat_templates import get_chat_template
from tqdm import tqdm  # Import tqdm for progress bar

def extract_llm_res(raw_llm):
    # Regex pattern to extract the assistant's response after the "Vulnerability Explanation" section
    pattern = r'\[/INST](.*?)</s>'

    match = re.search(pattern, raw_llm, re.DOTALL)

    if match:
        assistant_response = match.group(1).strip()
        return assistant_response
    else:
        print(f"No response found. Response: {raw_llm}")
        return raw_llm

# Set up the tokenizer and model for fast inference
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama",
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# Initialize the result collection
results = []

# Iterate over the dataset with tqdm for progress bar
for idx, example in tqdm(enumerate(test_dataset), total=len(test_dataset), desc="Processing dataset"):
    messages = example['conversations']  # Get the conversation for the current entry
    
    # Tokenize input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Get LLM output
    outputs = model.generate(input_ids=inputs, max_new_tokens=1024, use_cache=True, repetition_penalty=1.2,
                             temperature=0.1, min_p=0.1)
    raw_out = tokenizer.batch_decode(outputs)[0]
    
    # Extract LLM response
    llm_output = extract_llm_res(raw_out)
    
    # Get the ground truth for the current entry
    ground_truth = df_test[df_test['vuln_code'] == example['vuln_code']]['full_context'].values[0]
    
    results.append({
        "vuln_code": example['vuln_code'],
        "ground_truth": ground_truth,
        "llm_output": llm_output
    })

# After the loop, you can convert the results list to a DataFrame or another suitable format if needed
verification_results_df = pd.DataFrame(results)

Processing dataset:  73%|█████████████████████████████████████████████                 | 104/143 [1:32:24<40:36, 62.47s/it]

No response found. Response: <s> [INST] <<SYS>>
You are the smartest AI solidity smart contract security auditor in the world.
<</SYS>>

The given Solidity function is vulnerable. Review the code and analyze its security flaws. Just give short explanation why these function is vulnerable.
    
    This is the function we need to audit:
    ```solidity
    function setLockTime(uint256 newLockTime) external onlyOwner {
    require(newLockTime <= 7 days, "Lock too long");
    lockTime = newLockTime;
  }

function addLiquidity721(
    uint256 vaultId, 
    uint256[] memory ids, 
    uint256 minWethIn,
    uint256 wethIn
  ) public returns (uint256) {
    return addLiquidity721To(vaultId, ids, minWethIn, wethIn, msg.sender);
  }

function addLiquidity1155(
    uint256 vaultId, 
    uint256[] memory ids,
    uint256[] memory amounts,
    uint256 minWethIn,
    uint256 wethIn
  ) public returns (uint256) {
    return addLiquidity1155To(vaultId, ids, amounts, minWethIn, wethIn, msg.sender);
  

Processing dataset: 100%|██████████████████████████████████████████████████████████████| 143/143 [2:02:49<00:00, 51.54s/it]


In [ ]:
verification_results_df.to_csv("zeroshot_exp_codellama34b.csv", index=False)